In [ ]:
#If you have a GPU available, you can run the code using 
using CuArrays
gpu = true; CuArrays.allowscalar(false)

In [2]:
#otherwise set gpu to false
gpu = false;

In [30]:
using Dates, Plots             
using Logging, TerminalLoggers
global_logger(TerminalLogger());

In [3]:
import RetinalChaos: Network, extract_dict, read_JSON
import RetinalChaos: tar_conds, tar_pars

In [4]:
dt = 10.0; tspan = (0.0, 300e3)
nx = 96; ny = 96;

In [38]:
SACnet = Network(96, 96; μ = 0.70, gpu = gpu, version = :gACh)
p_dict = read_JSON("params.json");
u_dict = read_JSON("conds.json");
u0 = extract_dict(u_dict, tar_conds, (nx, ny));
p0 = extract_dict(p_dict, tar_pars);

In [39]:
import RetinalChaos: SDEProblem, noise_2D, SOSRI, solve

In [40]:
function run_simulation(prob; run_time = 300e3, warmup_time = 60e3, dt = 10.0)
    println("Warming up solution")
    prob = SDEProblem(prob.f, prob.g, prob.u0, (0.0, warmup_time), prob.p);
    sol = solve(
            prob,
            SOSRI(),
            abstol = 0.2,
            reltol = 2e-2,
            maxiters = 1e7,
            progress = true, 
            save_everystep = false
        )
    #Take the last solution to the ODE and use it as the initial conditions
    warmed_up_prob = SDEProblem(prob.f, prob.g, sol[end], (0.0, run_time), prob.p);
    sol = solve(
            warmed_up_prob,
            SOSRI(),
            abstol = 0.2,
            reltol = 2e-2,
            maxiters = 1e7,
            progress = true, 
            saveat = dt,
        )
    return sol
end

run_simulation (generic function with 1 method)

In [ ]:
prob = SDEProblem(SACnet, noise_2D, u0, (0.0, 1.0), p0);
sol = run_simulation(prob);

Warming up solution


SDE   0%|█                                              |  ETA: N/A
SDE   0%|█                                              |  ETA: 0:25:29
SDE   1%|█                                              |  ETA: 0:42:46
SDE   1%|█                                              |  ETA: 0:53:11
SDE   1%|█                                              |  ETA: 1:00:57
SDE   1%|█                                              |  ETA: 1:06:42
SDE   1%|█                                              |  ETA: 1:13:20
SDE   1%|█                                              |  ETA: 1:17:47
SDE   1%|█                                              |  ETA: 1:21:54
SDE   1%|█                                              |  ETA: 1:24:53
SDE   1%|█                                              |  ETA: 1:28:03
SDE   2%|█                                              |  ETA: 1:29:34
SDE   2%|█                                              |  ETA: 1:30:51
SDE   2%|█                                              |  ETA: 1:30

### [1.5.b] Graphing and visualization

In [ ]:
anim = @animate for i = 1:100:size(sol, 4)
    #println("Animating frame $i")
    p = plot(layout = grid(2, 2), size = (800, 800))
    heatmap!(p[1], sol[:, :, 1, i], ratio = :equal, grid = false,
        xaxis = "", yaxis = "", xlims = (0, nx), ylims = (0, ny),
        c = :curl, clims = (-70.0, 0.0),
    )
end

In [ ]:
gif(anim, "t_map.gif", fps = 5)

### [1.5.c] Analysis of the wave Data
- The output of the wave we get is very similar to the output of the trace simulation, except it is in 3D (x, y, d(Var)) with respect to time (t)

### [1.5.c] We can do multiple runs of this simulation loop. This allows us to 
- Perform repeated trials
- Alter a variable

I will only be doing one of these however, because these trials are costly. The below code should be performed only on a computer with a capable GPU, otherwise this will take a very long time. I will be altering the amount of acetylcholine released by a single SAC in $\mu M / ms$($\rho$)

In [ ]:
par = :ρ
n_trials = 10
par_range = LinRange(1.0, 10.0, n_trials)

In [ ]:
for val in par_range
    with_logger(TerminalLogger()) do
        #Warmup
        SDE_mat_prob = SDEProblem(SACnet, noise_2D, u0, (0.0, 60e3), p0);
        SDE_mat_sol = solve(
            SDE_mat_prob,
            SOSRI(),
            abstol = 0.2,
            reltol = 2e-2,
            maxiters = 1e7,
            progress = true, 
            saveat = dt,
        )
        #Run the trial
        u0_new = SDE_mat_sol[end]
        SDE_mat_prob = SDEProblem(SACnet, noise_2D, u0_new, tspan, p0);
        SDE_mat_sol = solve(
            SDE_mat_prob,
            SOSRI(),
            abstol = 0.2,
            reltol = 2e-2,
            maxiters = 1e7,
            progress = true, 
            saveat = dt,
        )
        #Perform the Data Analysis TODO
    end
end